In [1]:
import cv2 as cv
import numpy as np
import math


In [2]:
# Distance Formulae
def distance(x1,y1,x2,y2):
    dist=math.sqrt(math.fabs(x2-x1)**2 + math.fabs(y2-y1)**2)
    return dist

In [ ]:
def nothing(x):
    pass

barsWindow = 'Bars'
hl = 'H Low'
hh = 'H High'
sl = 'S Low'
sh = 'S High'
vl = 'V Low'
vh = 'V High'

# set up for video capture on camera 0
cap = cv.VideoCapture(0)

# create window for the slidebars
cv.namedWindow(barsWindow, flags = cv.WINDOW_AUTOSIZE)

# create the sliders
cv.createTrackbar(hl, barsWindow, 0, 179, nothing)
cv.createTrackbar(hh, barsWindow, 0, 179, nothing)
cv.createTrackbar(sl, barsWindow, 0, 255, nothing)
cv.createTrackbar(sh, barsWindow, 0, 255, nothing)
cv.createTrackbar(vl, barsWindow, 0, 255, nothing)
cv.createTrackbar(vh, barsWindow, 0, 255, nothing)

# set initial values for sliders
cv.setTrackbarPos(hl, barsWindow, 0)
cv.setTrackbarPos(hh, barsWindow, 179)
cv.setTrackbarPos(sl, barsWindow, 0)
cv.setTrackbarPos(sh, barsWindow, 255)
cv.setTrackbarPos(vl, barsWindow, 0)
cv.setTrackbarPos(vh, barsWindow, 255)

while(True):
    ret, frame = cap.read()
    frame = cv.GaussianBlur(frame, (5, 5), 0)
    
    # convert to HSV from BGR
    hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)

    # read trackbar positions for all
    hul = cv.getTrackbarPos(hl, barsWindow)
    huh = cv.getTrackbarPos(hh, barsWindow)
    sal = cv.getTrackbarPos(sl, barsWindow)
    sah = cv.getTrackbarPos(sh, barsWindow)
    val = cv.getTrackbarPos(vl, barsWindow)
    vah = cv.getTrackbarPos(vh, barsWindow)

    # make array for final values
    HSVLOW = np.array([hul, sal, val])
    HSVHIGH = np.array([huh, sah, vah])

    # apply the range on a mask
    mask = cv.inRange(hsv, HSVLOW, HSVHIGH)
    maskedFrame = cv.bitwise_and(frame, frame, mask = mask)

    # display the camera and masked images
    cv.imshow('Masked', maskedFrame)
    cv.imshow('Camera', frame)

	# check for q to quit program with 5ms delay
    if cv.waitKey(5) & 0xFF == ord('q'):
        break

# clean up our resources
cap.release()
cv.destroyAllWindows()

In [3]:
# RED(0,28,226) -> (66,243,255)
# GREEN(80,97,16) -> (102,236,239)

In [5]:
# Color1

def find_color1(frame):
    hsv_frame=cv.cvtColor(frame, cv.COLOR_BGR2HSV)
    hsv_lowerbound=np.array([0,28,226])
    hsv_upperbound=np.array([66,243,255])
    mask=cv.inRange(hsv_frame,hsv_upperbound,hsv_lowerbound)
    res=cv.bitwise_and(frame,frame,mask=mask)
    cnts,hir = cv.findContours(mask,cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    if len(cnts)>0:
        maxcontour=max(cnts,key=cv.contourArea)

        M=cv.moments(maxcontour)
        if M['m00']>0 and cv.contourArea(maxcontour)>1000:
            cx=int(M['m10']/M['m00'])
            cy=int(M['m01']/M['m00'])
            return(cx,cy),True
        else:
            return(700,700),False
    else:
        return(700,700),False

In [6]:
def find_color2(frame):
    """
    Filter "frame" for HSV bounds for color1 (inplace, modifies frame) & return coordinates of the object with that color
    """
    hsv_frame = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
    hsv_lowerbound =  np.array([80,97,16])
    hsv_upperbound = np.array([102,236,239])
    mask = cv.inRange(hsv_frame, hsv_lowerbound, hsv_upperbound)
    res = cv.bitwise_and(frame, frame, mask=mask)
    cnts, hir = cv.findContours(mask, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    if len(cnts) > 0:
        maxcontour = max(cnts, key=cv.contourArea)

        M = cv.moments(maxcontour)
        if M['m00'] > 0 and cv.contourArea(maxcontour) > 2000:
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            return (cx, cy), True
        else:
            return (700, 700), True
    else:
        return (700, 700), True

In [15]:
# Main Loop
vid=cv.VideoCapture(0)
# dummy1=cv.VideoCapture(0)
while(True):
    ret,dummy1=vid.read()
    # if frame is None:
    #     break 
    # dummy1=frame.copy()
    
    (color1_x, color1_y), found_color1 = find_color1(dummy1)
    (color2_x, color2_y), found_color2 = find_color2(dummy1)

    
    cv.circle(dummy1, (color1_x, color1_y), 20, (255, 0, 0), -1)
    cv.circle(dummy1, (color2_x, color2_y), 20, (0, 128, 255), -1)

    if found_color1 and found_color2:
        # getting line        
        hypotenuse = distance(color1_x, color1_y, color2_x, color2_y)
        horizontal = distance(color1_x, color1_y, color2_x, color1_y)
        vertical = distance(color2_x, color2_y, color2_x, color1_y)
        angle = np.arcsin(vertical/hypotenuse)*180.0/math.pi

        # line draw
        cv.line(dummy1, (color1_x, color1_y), (color2_x, color2_y), (0, 0, 255), 2)
        cv.line(dummy1, (color1_x, color1_y), (color2_x, color1_y), (0, 0, 255), 2)
        cv.line(dummy1, (color2_x, color2_y), (color2_x, color1_y), (0, 0, 255), 2)

        #angle text
        angle_text = ""
        if color2_y < color1_y and color2_x > color1_x:
            angle_text = str(int(angle))
        elif color2_y < color1_y and color2_x < color1_x:
            angle_text = str(int(180 - angle))
        elif color2_y > color1_y and color2_x < color1_x:
            angle_text = str(int(180 + angle))
        elif color2_y > color1_y and color2_x > color1_x:
            angle_text = str(int(360 - angle))

        cv.putText(dummy1, angle_text, (color1_x-30, color1_y), cv.FONT_HERSHEY_COMPLEX, 1, (0, 128, 229), 2)
    cv.imshow('Angle',dummy1)
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
    vid.release()
    cv.destroyAllWindows()

error: OpenCV(4.5.4-dev) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [9]:
# vid=cv.VideoCapture(1)
# while(True):
#     ret,frame=vid.read()
#     cv.imshow('frame',frame)
#     if cv.waitKey(1) & 0xFF == ord('q'):
#         break
# vid.release()
# cv.destroyAllWindows()